In [1]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)


2024-09-11 10:51:30.853180: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-11 10:51:30.860624: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-11 10:51:30.869838: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-11 10:51:30.872487: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-11 10:51:30.879326: I tensorflow/core/platform/cpu_feature_guar

TensorFlow version: 2.17.0


# Load dataset for test

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


# Build a machine learning model

In [3]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

/home/nata-brain/miniconda3/envs/tensor/lib/python3.9/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
I0000 00:00:1726062771.445383   83293 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1726062771.565081   83293 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1726062771.569059   83293 cuda_executor.cc:1015] successful NUMA 

In [4]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.11579275, -0.28881466, -0.10951762,  0.42449743, -0.36581236,
        -0.5283193 , -0.47982982, -0.156273  ,  0.11203833,  0.4870823 ]],
      dtype=float32)

In [5]:
tf.nn.softmax(predictions).numpy()

array([[0.0930829 , 0.07829382, 0.09366885, 0.15977727, 0.07249162,
        0.06161861, 0.06468009, 0.08939013, 0.11690031, 0.17009644]],
      dtype=float32)

In [6]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [7]:
loss_fn(y_train[:1], predictions).numpy()

2.7867913

In [8]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [9]:
model.fit(x_train, y_train, epochs=5)


Epoch 1/5


I0000 00:00:1726062852.290330   83760 service.cc:146] XLA service 0x7efd8c003c90 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1726062852.290347   83760 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-09-11 10:54:12.300657: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-09-11 10:54:12.334127: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


 237/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 649us/step - accuracy: 0.6944 - loss: 1.0355

I0000 00:00:1726062852.998515   83760 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 638us/step - accuracy: 0.8626 - loss: 0.4730
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 643us/step - accuracy: 0.9542 - loss: 0.1535
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 548us/step - accuracy: 0.9672 - loss: 0.1115
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 524us/step - accuracy: 0.9725 - loss: 0.0878
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 591us/step - accuracy: 0.9765 - loss: 0.0766


In [10]:
model.evaluate(x_test,  y_test, verbose=2)


313/313 - 1s - 3ms/step - accuracy: 0.9794 - loss: 0.0696


[0.06959166377782822, 0.9793999791145325]

In [11]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [12]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[2.7685240e-07, 8.1694038e-09, 3.9409085e-07, 9.3271256e-06,
        5.8262706e-10, 9.8751478e-08, 7.1629259e-13, 9.9998057e-01,
        1.8331941e-07, 9.0754975e-06],
       [5.9109323e-07, 9.1662523e-06, 9.9998593e-01, 2.4763131e-06,
        2.0622715e-18, 1.4407075e-08, 2.0343316e-09, 2.9889861e-14,
        1.8124853e-06, 5.0129477e-14],
       [2.8483743e-08, 9.9869388e-01, 4.9072671e-05, 2.5087722e-06,
        1.1513326e-04, 9.2781028e-07, 5.4372185e-07, 8.2890852e-04,
        3.0856926e-04, 4.6191127e-07],
       [9.9983561e-01, 3.7301877e-09, 1.5673527e-04, 1.3108298e-07,
        1.8333550e-07, 1.9743601e-07, 2.2305928e-06, 2.0983198e-06,
        3.5063419e-08, 2.6629168e-06],
       [9.2156256e-07, 9.4733721e-10, 6.2030567e-06, 5.6068330e-08,
        9.9682993e-01, 1.9899649e-07, 7.7277761e-08, 8.0882171e-05,
        9.3516144e-07, 3.0808311e-03]], dtype=float32)>